In [81]:
import sys
import os
sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../.."))
from create_fashion_items_image import create_cw_image
from fashion_class.ImageStruct import ImageStruct

import importlib
import fashion_class

from fashion_class.FashionItem import FashionItem
import gc
import random
from CreateCW import change_item_recommandation, search_alternate_item
from itertools import chain
importlib.reload(fashion_class.CapsuleWardrobeClass)
from fashion_class.CapsuleWardrobeClass import CapsuleWardrobe
import torch


In [83]:



class AllItemVersatilityCapsuleWardrobe(CapsuleWardrobe):
    def __init__(self, initial_items, target_items):
        super().__init__(initial_items=initial_items)
        self.NEAREST_ITEM_LENGTH = 100
        self.target_items = target_items

    def calc_versatility_increase(self, items: list[FashionItem], new_item: FashionItem):
        category = new_item.get_category()
        covered_item_ids = set()

        for i in items:
            X = i.img_tensor.expand(self.target_items[category].size())
             # ユークリッド距離を計算します
            distances = torch.sqrt(torch.sum((X - self.target_items[category]) ** 2, dim=1))
            # 上位k個の最小値のインデックスを取得
            sorted_indices = torch.argsort(distances)
            k_min_indices = sorted_indices[:self.NEAREST_ITEM_LENGTH]
            labels: set[int] = set(k_min_indices.tolist())
            covered_item_ids.union(labels)

        pre_score = len(covered_item_ids)
        X = new_item.img_tensor.expand(self.target_items[category].size())
        # ユークリッド距離を計算します
        distances = torch.sqrt(torch.sum((X - self.target_items[category]) ** 2, dim=1))
        # 上位k個の最小値のインデックスを取得
        sorted_indices = torch.argsort(distances)
        k_min_indices = sorted_indices[:self.NEAREST_ITEM_LENGTH]
        labels: list[int] = k_min_indices.tolist()
        covered_item_ids.union(labels)

        return (len(covered_item_ids) - pre_score) / self.NEAREST_ITEM_LENGTH

    def calc_self_cw_versatility(self):
        score = 0
        for items, category in zip([self.get_tops(), self.get_bottoms(), self.get_shoes()], ['tops', 'bottoms', 'shoes']):
            covered_item_ids = set()
            for i in items:
                X = i.img_tensor.expand(self.target_items[category].size())
                # ユークリッド距離を計算します
                distances = torch.sqrt(torch.sum((X - self.target_items[category]) ** 2, dim=1))
                # 上位k個の最小値のインデックスを取得
                sorted_indices = torch.argsort(distances)
                k_min_indices = sorted_indices[:self.NEAREST_ITEM_LENGTH]
                labels: set[int] = set(k_min_indices.tolist())
                covered_item_ids = covered_item_ids.union(labels)
            score += len(covered_item_ids)
        return score / (self.max_length * self.NEAREST_ITEM_LENGTH)

    def calc_compatibility_increase(self, _):
        return 0

    def calc_self_cw_compatibility(self):
        return 0
    
    def calc_image_simirality(self):
        score = 0
        for items, _ in zip([self.get_tops(), self.get_bottoms(), self.get_shoes()], ['tops', 'bottoms', 'shoes']):
            for i in range(self.max_length):
                for j in range(i+1, self.max_length):
                    score += torch.sqrt(torch.sum((items[i].img_tensor - items[j].img_tensor) ** 2, dim=1))
        return score

class ClusterCapsuleWardrobe(CapsuleWardrobe):
    def __init__(self, initial_items, target_items):
        super().__init__(initial_items=initial_items)
        self.target_tops = target_items['tops']
        self.target_bottoms = target_items['bottoms']
        self.target_shoes = target_items['shoes']

    def calc_compatibility_increase(self, _):
        return 0

    def calc_self_cw_compatibility(self):
        return 0

    def calc_image_simirality(self):
        score = 0
        for items, _ in zip([self.get_tops(), self.get_bottoms(), self.get_shoes()], ['tops', 'bottoms', 'shoes']):
            for i in range(self.max_length):
                for j in range(i+1, self.max_length):
                    score += torch.sqrt(torch.sum((items[i].img_tensor - items[j].img_tensor) ** 2, dim=1))
        return score

    


In [67]:
import copy

def create_cw(cw: CapsuleWardrobe, dataset, eps=1e-3):
    increase = eps + 1
    pre_cw = copy.deepcopy(cw)
    while increase > eps:
        # ここ遅い。topsとかカテゴリごとに、キャッシュしてあげたほうがいい。
        increase = cw.optimize(dataset)
        pre_cw = copy.deepcopy(cw)
        if increase < 0:
            cw = pre_cw
    return cw

In [6]:
annotation_file, tensor_file = \
        'D:/M1/fashion/optimization/data/anotation_new.csv', 'D:/M1/fashion/optimization/data/resnet50_tensor.pt'

dataset = ImageStruct(annotation_file, tensor_file, init_item_length=500)
target_items = {
    'tops':torch.stack([i.img_tensor for i in dataset.get_tops()]),
    'bottoms':torch.stack([i.img_tensor for i in dataset.get_bottoms()]),
    'shoes':torch.stack([i.img_tensor for i in dataset.get_shoes()]),
}

initial_items = {
    'tops': random.sample(dataset.get_tops(), 20),
    'bottoms':random.sample(dataset.get_bottoms(), 20),
    'shoes':random.sample(dataset.get_shoes(), 20),
}


【==================================================================================================  】

In [85]:

aivcw = AllItemVersatilityCapsuleWardrobe(initial_items, target_items)
ccw = ClusterCapsuleWardrobe(initial_items, target_items)
cw = CapsuleWardrobe(initial_items=initial_items)
aivcw = create_cw(aivcw, dataset)
ccw = create_cw(ccw, dataset)
cw = create_cw(cw, dataset)


In [86]:
aivcw.calc_self_cw_compatibility(), aivcw.calc_self_cw_versatility()

(0, 0.56)

In [87]:
ccw.calc_self_cw_compatibility(), ccw.calc_self_cw_versatility()

(0, 1.0)

In [70]:
image = create_cw_image(ccw)
image.show()

In [71]:
image = create_cw_image(aivcw)
image.show()